<a href="https://colab.research.google.com/github/jgomezpe/primenumbers/blob/main/lifrac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>Defining Upper and Lower Bounding Functions of $li(x)$ with $\displaystyle O\left(\sqrt{\frac{x}{\log(x)}}\right)$ Error Using Truncated Asymptotic Series</h2>

This program generates the computational results of Tables 1, 2, and 3 of the paper "Defining Upper and Lower Bounding Functions of $li(x)$ with $\displaystyle O\left(\sqrt{\frac{x}{\log(x)}}\right)$ Error Using Truncated Asymptotic Series" by <b>Professor Jonatan Gomez</b> from <b>Universidad Nacional de Colombia</b>.

In [ ]:
import math
from decimal import *

# Quantity of prime numbers up to 10^k (k position in the list)
# Taken from https://oeis.org/A006880 and
# https://en.wikipedia.org/wiki/Prime-counting_function
pi = [4, 25, 168, 1229, 9592, 78498, 664579, 5761455, 50847534, 455052511, 4118054813, 37607912018, 346065536839, 3204941750802, 29844570422669, 279238341033925, 2623557157654233, 24739954287740860, 234057667276344607, 2220819602560918840, 21127269486018731928, 201467286689315906290, 1925320391606803968923, 18435599767349200867866, 176846309399143769411680, 1699246750872437141327603,
      16352460426841680446427399, 157589269275973410412739598, 1520698109714272166094258063]
PI = [Decimal(x) for x in pi]


# Difference between li function and pi function
li_pi = [2.2, 5.1, 10, 17, 38, 130, 339, 754, 1701, 3104, 11588, 38263, 108971, 314890, 1052619, 3214632, 7956589, 21949555, 99877775, 222744644, 597394254, 1934355208, 7250186216, 17146907278, 55160980939, 155891678121, 508666658006, 1427745660374, 4551193622464]

# li function table 10^{i+1}
li = [li_pi[i]+pi[i] for i in range(len(pi))]
LI = [Decimal(x) for x in li]

def getKappa(omega, a, b):
  m = (a+b)/2
  while(abs(b-a) >= 1e-10):
    m = (a+b)/2
    fm = m*(1-math.log(m)) - omega
    if(abs(fm) < 1e-10):
      return m
    if(fm > 0.0):
      b = m
    else:
      a = m
  return m

def kappa_minus(omega):
  return getKappa(omega, 0, 1)

def kappa_plus(omega):
  return getKappa(omega, math.e, 1)

#Kappa Approximation series li(x, omega)
def plus_sum(ln_x, omega, alpha):
  kappa = Decimal(kappa_plus(omega))
  n = int(ln_x)
  m = int(kappa*ln_x)
  s = Decimal(1)
  p = Decimal(1)
  for k in range(1,m):
    p *= Decimal(Decimal(k)/ln_x)
    s += p
  if(alpha != 1):
    alpha = kappa*ln_x - Decimal(m)
  f = alpha * p * Decimal(Decimal(m)/ln_x)
  return s + f

def li_plus(x, omega, alpha=None):
  ln_x = Decimal.ln(x)
  return x*plus_sum(ln_x, omega, alpha)/ln_x

def minus_sum(ln_x, omega, alpha):
  kappa = Decimal(kappa_minus(omega))
  m = int(kappa*ln_x)
  if(alpha != 0):
    alpha = kappa*ln_x - Decimal(m)
  if(m==0): return alpha
  s = Decimal(1)
  p = Decimal(1)
  for k in range(1,m):
    p *= Decimal(Decimal(k)/ln_x)
    s += p
  f = alpha * p * Decimal(Decimal(m)/ln_x)
  return s + f

def li_minus(x, omega, alpha=None):
  ln_x = Decimal.ln(x)
  return x*minus_sum(ln_x, omega, alpha)/ln_x

def round2(x):
  x = str(int(x*Decimal(100))/100)
  if(x.find('.')==-1): x += '.00'
  while(x.find('.') > len(x)-3):
    x += '0'
  return x

def plus_frac_1_2(n):
  for i in range(2, n+1):
    s = plus_sum(i+1, 0.5)
    d = s - Decimal(1) - Decimal(1/(i+1)) - Decimal(2/(i+1)**2)
    d *= Decimal(i**2/2)
    print( i, s, d )


#main program
omega = 0.5
k_m = Decimal(kappa_minus(omega))
k_p = Decimal(kappa_plus(omega))
D_m = (Decimal(8)*Decimal(math.pi)/k_m)**(Decimal(0.5)) * Decimal.exp(Decimal(13/12)) *(Decimal(1)+k_m)/(Decimal(1)-k_m)
D_p = (Decimal(2)*Decimal(math.pi)/k_p)**(Decimal(0.5)) * Decimal.exp(Decimal(13/12)) *(Decimal(3)+k_p)/(k_p-Decimal(1))

print('$\\underline{\\kappa} \\approx ', k_m, '$', sep='')
print('$\\overline{\\kappa} \\approx ', k_p, '$', sep='')
print('$\\underline{D} \\approx ', D_m, '$', sep='')
print('$\\overline{D} \\approx ', D_p, '$', sep='')

x = Decimal(10)
LM = []
LP = []
DM = []
DP = []
root = []
for i in range(len(li)):
  LM.append(li_minus(x, omega, 0))
  LP.append(li_plus(x, omega, 1))
  DM.append(LI[i]-LM[i])
  DP.append(LP[i]-LI[i])
  root.append(Decimal.sqrt(x/Decimal.ln(x)))
  x *= Decimal(10)

latex = True
print("Table 1")
for i in range(len(li)):
  if(latex):
    print('$10^{', i+1, '}$ & $', round2(DM[i]), '$ & $', round2(DM[i]/root[i]), '$ & $', round2(DP[i]), '$ & $', round2(DP[i]/root[i]), '$ \\\\', sep='')
  else:
    print('10^', i+1, round2(DM[i]), round2(DM[i]/root[i]), round2(DP[i]), round2(DP[i]/root[i]))

x = Decimal(10)
LM = []
LP = []
DM = []
DP = []
root = []
for i in range(len(li)):
  LM.append(li_minus(x, omega))
  LP.append(li_plus(x, omega))
  DM.append(LI[i]-LM[i])
  DP.append(LP[i]-LI[i])
  root.append(Decimal.sqrt(x/Decimal.ln(x)))
  x *= Decimal(10)

print("Table 2")
for i in range(len(li)):
  if(latex):
    print('$10^{', i+1, '}$ & $', round2(DM[i]), '$ & $', round2(DM[i]/root[i]), '$ & $', round2(DP[i]), '$ & $', round2(DP[i]/root[i]), '$ \\\\', sep='')
  else:
    print('10^', i+1, round2(DM[i]), round2(DM[i]/root[i]), round2(DP[i]), round2(DP[i]/root[i]))

print("Table 3")
for i in range(len(li)):
  if(latex):
    print('$10^{', i+1, '}$ & $', round2(PI[i]-LM[i]), '$ & $', round2(LP[i]-PI[i]), '$ \\\\', sep='')
  else:
    print('10^', i+1, round2(PI[i]-LM[i]), round2(LP[i]-PI[i]))



$\underline{\kappa} \approx 0.186682308907620608806610107421875$
$\overline{\kappa} \approx 2.15553520341404070137514281668700277805328369140625$
$\underline{D} \approx 50.01822612661667099964767375$
$\overline{D} \approx 22.50549858928504234579802843$
Table 1
$10^{1}$ & $6.20$ & $2.97$ & $7.50$ & $3.60$ \\
$10^{2}$ & $30.10$ & $6.45$ & $19.04$ & $4.08$ \\
$10^{3}$ & $33.23$ & $2.76$ & $48.41$ & $4.02$ \\
$10^{4}$ & $160.26$ & $4.86$ & $131.96$ & $4.00$ \\
$10^{5}$ & $189.66$ & $2.03$ & $371.69$ & $3.98$ \\
$10^{6}$ & $1006.37$ & $3.74$ & $1081.41$ & $4.01$ \\
$10^{7}$ & $1228.85$ & $1.56$ & $3214.43$ & $4.08$ \\
$10^{8}$ & $6824.92$ & $2.92$ & $9688.79$ & $4.15$ \\
$10^{9}$ & $41026.02$ & $5.90$ & $29513.14$ & $4.24$ \\
$10^{10}$ & $48256.77$ & $2.31$ & $90615.57$ & $4.34$ \\
$10^{11}$ & $290952.54$ & $4.63$ & $279938.15$ & $4.45$ \\
$10^{12}$ & $349620.27$ & $1.83$ & $869086.04$ & $4.56$ \\
$10^{13}$ & $2111307.71$ & $3.65$ & $2709093.04$ & $4.68$ \\
$10^{14}$ & $2574265.41$ & $1.46$